<a href="https://colab.research.google.com/github/achri19/komo_estuary_tutorial/blob/main/1_GEEwatermask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="5">Build Your Own ANUGA Model (BYOAM) </font>

<font size="3">In this notebook, we will:

- define our study area

- build a working directory


- initiate the Google Earth Engine process for building water masks</font>

<font size="3">This notebook is not required because it will be slow, requires Google Earth Engine account, and is not related to the ANUGA model. Therefore, you don't need to run this tutorial, the output files will be provided in the next tutorial. </font> <br></br>
<font size="3">This could take several hours on GEE, depending on the size of the study area. Once the four water masks are made, you must choose which one is best. There are more details below.</font>



<font size=5> Step #1: Mount Google Drive and Grant Access <font> <br>




<font size=3> When you run the next cell, a pop-up window will appear asking you to grant access to your Google Drive. You must approve or the notebook will not work. <font> <br>


In [1]:
import sys
import os
path = os.getcwd() + '/'
where_to_mount = '/content/drive/'

from google.colab import drive
drive.mount(where_to_mount, force_remount=True)

mounted_drive = where_to_mount + 'MyDrive/'

Mounted at /content/drive/


<font size=5> Step #2: Install packages. <font> <br>
<font size=3> This cell should install all Python packages you need for each tutorial.<font>

In [2]:
import os
os.chdir(mounted_drive)
if os.path.isdir(mounted_drive + 'komo_estuary_tutorial'):
  print('## Updating the local git repository \n')
  os.chdir(mounted_drive + '/komo_estuary_tutorial')
  ! git pull 
else:
  print('## Pulling the git repository with files for the tutorial\n')
  ! git clone https://github.com/achri19/komo_estuary_tutorial.git

print('\n## Installing the Python packages needed for these tutorials\n')
!/bin/bash $mounted_drive/komo_estuary_tutorial/install_packages.sh

if os.path.isdir(mounted_drive + 'komo') == False:
  !unzip komo.zip -d $mounted_drive 


## Updating the local git repository 

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/achri19/komo_estuary_tutorial
   54efd8a..ee44e58  main       -> origin/main
Updating 54efd8a..ee44e58
Fast-forward
 0_SetupColab.ipynb | 265 ++++++++++++++++++++++++-----------------------------
 1 file changed, 122 insertions(+), 143 deletions(-)

## Installing the Python packages needed for these tutorials

(1) Install pip packages
nose mpi4py triangle Pmw pymetis cmocean geopandas fiona pygeos rasterio rasterstats
(2) Install gdal
(3) Install netcdf4
(4) Download anuga_core github repository
https://github.com/GeoscienceAustralia/anuga_core
(5) Install anuga
(7) Completed


<font size=5> Step #3: Set up working directories<font> 


<font size=3> We will access data files stored on a shared Google Drive. You will also need to save files to your own Google Drive.<font>



In [ ]:
path_code = where_to_mount + 'MyDrive/processing/code/'
path_templates = where_to_mount + 'MyDrive/processing/templates/'
path_configs = where_to_mount + 'MyDrive/processing/configs/'
path_ancillary = where_to_mount + 'MyDrive/processing/ancillary/'
try:
  os.mkdir(path_examples)
except:''
sys.path.insert(1,path_code)


<font size=5> Step #4: Now we will import Python packages, which will be used for future tutorials. If there are any errors, try rerunning the first cell <font> 




In [ ]:
# Standard packages
import fnmatch
from pathlib import Path
import geopandas as gpd

## Plotting modules
import matplotlib.pyplot as plt

## Google Earth Packages
import ee
from ee import batch
import geemap


In [ ]:
sys.path.insert(1,path_code)
from BYOM_Utilities_V1 import (build_directory)

/content/drive/MyDrive/processing/code/raster_bridges.py:39: UserWarning: ShapelyDeprecationWarning
  warnings.warn('ShapelyDeprecationWarning')
/content/drive/MyDrive/processing/code/raster_bridges.py:40: UserWarning: UserWarning
  warnings.warn('UserWarning')


<font size=3 color='red' > Give a name to your model area <font>

In [ ]:
AOI = 'komo'

<font color='red' size="5">Now we can start buiding the model files</font>
- <font color='red' size="3"> Choose a name for your study area and build the working directory. All files for the model will be saved within the folder "EXAMPLES"</font>


In [ ]:
Path((path_examples + AOI)).mkdir(parents=True, exist_ok=True)
working_path,folders = build_directory(path_examples, AOI)




##############################################################################################
################################[Step 1][Build Directory]#####################################
##############################################################################################

##################### The working directory set as: 

/content/drive/MyDrive/examples/komo/
 
##################### Folders are:
##################### 0 User_Defined_Files --> User shapefile of model domain and water mask
##################### 1 tmp --> For temporary files
##################### 3 Meshes --> Where we will build model meshes
##################### 4 DEMs --> Where we will build digital elevation models
##################### 5 Boundaries --> Where we will store boundary files
##################### 6 Simulations --> Where we will run simulations
##################### 7 Setup_Files/Setup_SHP --> Shapefiles for setup 
##################### 8 Setup_Files/Setup_RST --> Rasters for setup
#######

<font size="5" color='red'> The model domain is defined with a shapefile. If you have a shapefile to use, move it to the folder __/User_Defined_Files/__ and rename all of the associated files (.shp, .dbf, .fix, .prj, .shx, .cpg) as **_input where ** is the name of your AOI.</font>


<font size="5" color='red' > Now we will initiate Google Earth Engine (python API) to build a water mask. You may need to authenticate Google. </font>

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=QP0mFwYbWPdHtSV2cXiNYbTeFUDEtkfpm2L8HgT5iho&tc=jtHY-eAZL-aOuem-d25DJPI1RN8mYy9_64_Oy-Vbydg&cc=e9WZqdjpYB89OwYqDAeVXk5wAbe2kI_hQ2MCLNmEyBE

The authorization workflow will generate a code, which you should paste in the box below.


KeyboardInterrupt: ignored

In [ ]:
## Create a map for defining area of interest
Map = geemap.Map()
Map

<font size="3">And define a few Google Earth Engine Functions </font>

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60');
    ##Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    ##Both flags should be set to zero, indicating clear conditions.
    cloudmask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    #cloudmask = qa.bitwiseAnd(cloudBitMask).eq(0)*(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(cloudmask)#.divide(10000)
    #return image.updateMask(qa.bitwiseAnd(cloudBitMask).eq(0)).updateMask(qa.bitwiseAnd(cirrusBitMask).eq(0)).divide(100000)  
def addNDWI(image):
  ndwi = image.normalizedDifference(['B8', 'B3']).rename('NDWI').toFloat();
  return image.addBands(ndwi)    
def addNDVI(image):
  #ndvi = image.normalizedDifference(['B8', 'B3']).rename('NDVI').toFloat();
  ndvi = (image.select(['B8']).subtract(image.select(['B3']))).divide(image.select(['B8']).add(image.select(['B3']))).rename('NDVI')
  return image.addBands(ndvi)
def addMNDWI(image):
  mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI');
  return image.addBands(mndwi)

<font size="5" color='red'> If you don't have a shapefile, zoom to your study area in the map above and use the rectangle icon to draw a boundary around your study area. This will be the model domain..</font>


In [ ]:
extent_file  = folders[0] + '/' + AOI + '_input.shp'

if os.path.isfile(extent_file)==True:
    print('Model domain defined by %s' %(extent_file))
    ee_object = geemap.shp_to_ee(extent_file)
    Map.addLayer(ee_object, {}, 'Layer name')
    Map.centerObject(ee_object,7)
else: 
    print('You must define the model domain using the map above')
    Map.draw_last_feature
    roi = ee.FeatureCollection(Map.draw_features)
    geemap.ee_export_vector(roi, folders[0] + '/' + AOI + '_input.shp')

<font size="5" color='red' >If you got an error above, go back to the map and draw (rectangle icon) a boundary around your study area and then rerun the previous cell </font>

<font size = 5> If you want to try generating water masks in GEE, change the run_gee to True and the remaining cells will submit tasks to Google Earth Engine.  </font>


In [ ]:
run_gee = False


<font size="5" color='red' > The next cell ensures the projections are correct and defines the extent in Google Earth Engine.  </font>
    

In [ ]:
extent_gpd = gpd.read_file(extent_file)
resolution  = 10 ## resolution of the water mask

if extent_gpd.crs == None:
    extent_gpd = extent_gpd.set_crs('EPSG:4326')
if extent_gpd.crs != 'EPSG:4326':
    print('reprojecting to EPSG')
    extent_gpd = extent_gpd.to_crs('EPSG:4326')

extent_gpd.geometry = extent_gpd.buffer(0.2,join_style = 2)
exterior_points = extent_gpd.geometry[0].exterior.coords[:]
exterior_points = list(map(list, exterior_points))
ee_poly = ee.Geometry.Polygon(coords=exterior_points)    

Map.addLayer(ee_poly)

<font size='5' color = 'red' > Now, we collect Sentinel-1 and Sentinel-2 data. </font>
- <font size='3' color='red' > All dates for Sentinel-1 data are used and both polarizations (VV and VH) </font>
- <font size='3' color='red' > All dates for Sentinel-2 data are used, but are filtered to only images with  less than 5% cloudy fixels. We add NDVI, NDWI, and a cloud mask </font>


In [ ]:
collectionS2 = ee.ImageCollection("COPERNICUS/S2")\
    .filter(ee.Filter.calendarRange(2019,2022,'year'))\
    .filter(ee.Filter.calendarRange(1,12,'month'))\
    .filterBounds(ee_poly)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))\
    .map(addNDWI)\
    .map(addMNDWI)\
    .map(addNDVI)\
    .map(maskS2clouds)  ;  
collectionVV = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.calendarRange(2019,2022,'year'))\
    .filter(ee.Filter.calendarRange(1,12,'month'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filterMetadata('resolution_meters', 'equals' , 10)\
    .filterBounds(ee_poly)\
    .select('VV');
collectionVH = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.calendarRange(2019,2022,'year'))\
    .filter(ee.Filter.calendarRange(1,12,'month'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filterMetadata('resolution_meters', 'equals' , 10)\
    .filterBounds(ee_poly)\
    .select('VH');

<font size='5' color = 'red' > Now, process those collections, further. </font>
- <font size='3' color='red' > For the Sentinel-1 VV and VH collections, for each pixel, we take the maximum value within the stack. Then we find the focal mean within 30m radius to remove speckle/noise </font>
- <font size='3' color='red' > For the Sentinel-2 collection, we take the min, max, and mean of the all bands.</font>


In [ ]:
## Process Sentinel-1 Data
VV_ref = collectionVV.max().focal_mean(30, 'circle', 'meters')
VH_ref = collectionVH.max().focal_mean(30, 'circle', 'meters')

# Process Sentinel-2 Data
S2_max = collectionS2.max()    
S2_min = collectionS2.min()
S2_mean = collectionS2.mean()    
ndwi_min = S2_min.select('NDWI')
ndvi_min = S2_min.select('NDVI')   
ndwi_max = S2_max.select('NDWI')
ndvi_max = S2_max.select('NDVI')   
ndwi_mean = S2_mean.select('NDWI')
ndvi_mean = S2_mean.select('NDVI')   

folder_on_drive = 'gee_' + AOI

<font size='5' color = 'red' > We use four different alogirthms to create waste masks. With each set of inputs, we use the GEE Kmeans algorithm to cluster pixels into 2 classes, which we later classify as water or not water. </font>
- <font size='3' color='red' > (1) Max NDWI, max VV ,and max VH </font>
- <font size='3' color='red' > (2) Mean NDWI, max VV, and max VH </font>
- <font size='3' color='red' > (3) Max NDWI and max NDVI </font>
- <font size='3' color='red' > (4) Min NDWI and min NDVI </font>

<font size='3' color='red' > Once you run the 4 cells below, the tasks are submitted to GEE. If you run them multiple times, multiple duplicate tasks will be submitted. </font>

In [ ]:
if run_gee:

    ## NDWI max, VV, and VH composite
    bands = ['NDWI','VV','VH'];
    ref_composite = ee.Image.cat(ndwi_max.toFloat(),VV_ref.toFloat(),VH_ref.toFloat()).select(bands).clip(ee_poly);
    method = 'ndwimax_vvvhmax'
    print('Method is %s' %(method))

    ## Save composite
    # comp_out1 = batch.Export.image.toDrive(ref_composite, folder = folder_on_drive, description='%s_%s' %(AOI,method), fileFormat = 'GeoTiff',scale = 10,maxPixels=10000000000000 )
    # process = batch.Task.start(comp_out1)
    # print('Saved composite to Google Drive at GEE_drive_composites/%s_%s.tif' %(AOI,method))

    ## Unsupervised Clustering using S2 max, VV, and VH composite
    training = ref_composite.sample(region=ee_poly,scale=resolution,numPixels=500,tileScale = 16)
    classifier = ee.Clusterer.wekaKMeans(2).train(training,bands)
    water = ref_composite.select(bands).cluster(classifier);

    ## Save clusters
    mask_out1 = batch.Export.image.toDrive(water, region=ee_poly,folder = folder_on_drive, description='%s_%s_clustered' %(AOI,method),fileFormat = 'GeoTiff',scale = 10,maxPixels=10000000000000 )
    process = batch.Task.start(mask_out1)
    print('Saved clusters to Google Drive at %s/%s_%s' %(folder_on_drive,AOI,method))

else:
    print('Skipping')

In [ ]:
if run_gee:
    ## Save S2 mean VV and VH max
    bands = ['NDWI','VV','VH'];
    ref_composite = ee.Image.cat(ndwi_mean.toFloat(),VV_ref.toFloat(),VH_ref.toFloat()).select(bands).clip(ee_poly);
    method = 'ndwimean_vvvhmax'
    print('Method is %s' %(method))

    ## Save composite
    # comp_out2 = batch.Export.image.toDrive(ref_composite, folder = folder_on_drive, description='%s_%s' %(AOI,method), fileFormat = 'GeoTiff',scale = 10,maxPixels=10000000000000 )
    # process = batch.Task.start(comp_out2)
    # print('Saved composite to Google Drive at GEE_drive_composites/%s_%s.tif' %(AOI,method))

    ## Unsupervised Clustering using S2 max, VV, and VH composite
    training = ref_composite.sample(region=ee_poly,scale=resolution,numPixels=500,tileScale = 16)
    classifier = ee.Clusterer.wekaKMeans(2).train(training,bands)
    water = ref_composite.select(bands).cluster(classifier);

    ## Save clusters
    mask_out2 = batch.Export.image.toDrive(water, region=ee_poly,folder = folder_on_drive, description='%s_%s_clustered' %(AOI,method),fileFormat = 'GeoTiff',scale = 10,maxPixels=10000000000000 )
    process = batch.Task.start(mask_out2)
    print('Saved clusters to Google Drive at GEE_drive_masks/%s_%s' %(AOI,method))

else:
    print('Skipping')


In [ ]:
if run_gee: 
    ## Save NDVI and NDWI min
    bands = ['NDVI','NDWI'];
    ref_composite = ee.Image.cat(ndvi_min.toFloat(),ndwi_min.toFloat()).select(['NDWI','NDVI']).clip(ee_poly);      
    method = 'ndvimin_ndwimin'
    print('Method is %s' %(method))

    ## Save composite
    # comp_out3 = batch.Export.image.toDrive(ref_composite, folder = folder_on_drive, description='%s_%s' %(AOI,method), fileFormat = 'GeoTiff',scale = 10,maxPixels=10000000000000 )
    # process = batch.Task.start(comp_out3)
    # print('Saved composite to Google Drive at GEE_drive_composites/%s_%s.tif' %(AOI,method))

    ## Unsupervised Clustering using S2 max, VV, and VH composite
    training = ref_composite.sample(region=ee_poly,scale=resolution,numPixels=500,tileScale = 16)
    classifier = ee.Clusterer.wekaKMeans(2).train(training,bands)
    water = ref_composite.select(bands).cluster(classifier);

    ## Save clusters
    mask_out3 = batch.Export.image.toDrive(water, region=ee_poly,folder = folder_on_drive, description='%s_%s_clustered' %(AOI,method),fileFormat = 'GeoTiff',scale = 10,maxPixels=10000000000000 )
    process = batch.Task.start(mask_out3)
    print('Saved clusters to Google Drive at GEE_drive_masks/%s_%s' %(AOI,method))

else:
    print('Skipping')

In [ ]:
if run_gee:
    ## Save NDVI and NDWI max
    bands = ['NDVI','NDWI'];
    ref_composite = ee.Image.cat(ndvi_max.toFloat(),ndwi_max.toFloat()).select(['NDWI','NDVI']).clip(ee_poly);      
    method = 'ndvimax_ndwimax'
    print('Method is %s' %(method))

    ## Save composite
    #comp_out4 = batch.Export.image.toDrive(ref_composite, folder = folder_on_drive, description='%s_%s' %(AOI,method), fileFormat = 'GeoTiff',scale = 10,maxPixels=10000000000000 )
    #process = batch.Task.start(comp_out4)
    #print('Saved composite to Google Drive at GEE_drive_composites/%s_%s.tif' %(AOI,method))

    ## Unsupervised Clustering using S2 max, VV, and VH composite
    training = ref_composite.sample(region=ee_poly,scale=resolution,numPixels=500,tileScale = 16)
    classifier = ee.Clusterer.wekaKMeans(2).train(training,bands)
    water = ref_composite.select(bands).cluster(classifier);

    ## Save clusters
    mask_out4 = batch.Export.image.toDrive(water, region=ee_poly,folder = folder_on_drive, description='%s_%s_clustered' %(AOI,method),fileFormat = 'GeoTiff',scale = 10,maxPixels=10000000000000 )
    process = batch.Task.start(mask_out4)
    print('Saved clusters to Google Drive at GEE_drive_masks/%s_%s' %(AOI,method))

else:
    print('Skipping')


<font size='5' color = 'red' > The tasks are now submitted to GEE. These jobs will take a while (less than 1 hour for small areas or many hours for large areas)</font>
- <font size='3' color='red' > You can check the status by going to https://code.earthengine.google.com/tasks</font>
- <font size='3' color='red' > We will also monitor the status below, but it will take time. </font>



<font size='5' color = 'red' > Once the tasks have completed, you must download them. Go to your [drive.google.com](https://drive.google.com) and download them manually. They should be saved in a folder called __gee_komo__. However, it could take a few hours before they are ready.</font>


<font size='5' color = 'red' >  Finally, you must choose the best one. </font>

<font size='3' color = 'red' > Use QGIS to pick the best water mask. Add __finalwatermask__ to the end of the file name. The file ending in __finalwatermask.tif__ will be used in future notebooks. </font>


<font size='5' color = 'red' > This notebook is complete, continue to notebook #2 to build the model foundations </font>
